In [8]:
import uproot
import matplotlib
import matplotlib.pyplot as plt
import awkward as ak
import hist
import numpy as np
from event_display import gdml_read_ecal_info
import os
import pandas as pd
import matplotlib.patches as mpatches
from scipy.stats import crystalball, norm

%matplotlib inline
matplotlib.rc("figure", dpi=200)

dir_path = os.path.dirname(os.path.realpath("__file__"))
print(f"Believed path is: {dir_path}")

Believed path is: /home/romanov/eic/kak-bog-cherepahu/analysis


In [9]:
import pandas as pd

all_vals_df = pd.read_feather("/mnt/work/data/epi/2022-12-02_dd4hep_SciGlass_13x13_20x20x400mm_0-10GeV_e-pi-_100kev.feather")
all_vals_df["e_div_p"] = all_vals_df.de_sum/(all_vals_df.p)
all_vals_df.head()

,p,prt_name,de_sum,towers,e_div_p
0,0.49,e-,0.671115,"[0, 0]",1.369623
1,0.49,e-,0.675113,"[0, 0]",1.377782
2,0.49,e-,0.674566,"[0, 0]",1.376666
3,0.49,e-,0.658135,"[0, 0]",1.343133
4,0.49,e-,0.668929,"[0, 0]",1.365162


In [10]:
def make_2sigma_separation(e_df, pi_df):
    # Fit with crystalball and gauss
    cb_beta, cb_m, cb_loc, cb_scale = crystalball.fit(e_df.e_div_p, loc=0.6)
    norm_loc, norm_scale = norm.fit(e_df.e_div_p)
    print(cb_beta, cb_m, cb_loc, cb_scale)
    print(norm_loc, norm_scale)

    # where we separate
    separ_x = cb_loc - norm_scale*3
    n_pi_above_num = np.size(pi_df.query(f"e_div_p > {separ_x}").e_div_p)
    n_e_below_num = np.size(e_df.query(f"e_div_p <= {separ_x}").e_div_p)
    n_e_above_num = np.size(e_df.query(f"e_div_p > {separ_x}").e_div_p)

    n_pi_above = n_pi_above_num / np.size(pi_df.e_div_p)
    n_e_below = n_e_below_num / np.size(e_df.e_div_p)
    n_e_above = n_e_above_num / np.size(e_df.e_div_p)

    print(separ_x, n_pi_above, n_e_below, n_e_above, np.size(e_df.e_div_p))

    # We can set the number of bins with the *bins* keyword argument.
    fig, axs = plt.subplots()    
    axs.hist(e_df.e_div_p, bins=50, alpha = 0.5, edgecolor='blue', linewidth=0.3, range=(0, 1), density=True, label="$e^{-}$")
    axs.hist(pi_df.e_div_p, bins=50, alpha = 0.5, edgecolor='red', linewidth=0.3, range=(0, 1), density=True, label="$\pi^{-}$")
    axs.axvline(x=separ_x, color='red', linestyle='--', linewidth=0.8, label=f"peak-3$\sigma$ = {separ_x:.2f}")
    axs.plot([], [], ' ', label="$\epsilon_{e^{-}}$ = " + f"{n_e_above*100:.2f}%")
    axs.plot([], [], ' ', label="$\epsilon_{\pi^{-}}$ = " + f"{n_pi_above*100:.2f}%")
    axs.legend(loc='best')
    # txt = ax.text(0.5, 0.5, f"{separ_x}\n {n_pi_above}, {n_e_below}, {n_e_above}")
    # txt.set_clip_on(False)
    # handles, labels = plt.get_legend_handles_labels()
    # handles.append(mpatches.Patch(color='none', label=extraString))
    # plt.legend(handles=handles)
    # # txt.set_clip_on(False)  # this will turn clipping off (always visible)
    # plt.plot()

def df_make_separation(df, e_center, range=1):
    e_min = e_center - 0.1*range
    e_max = e_center + 0.1*range
    e_filtered_df = all_vals_df.query(f"({e_min} < p < {e_max}) and prt_name=='e-'")
    pi_filtered_df = all_vals_df.query(f"({e_min} < p < {e_max}) and prt_name=='pi-'")

    make_2sigma_separation(e_filtered_df, pi_filtered_df)


In [11]:
df_make_separation(all_vals_df, 0.5, 0.1)


/usr/local/lib/python3.8/dist-packages/scipy/stats/_distn_infrastructure.py:2603: RuntimeWarning: Mean of empty slice.
  muhat = tmp.mean()
/usr/local/lib/python3.8/dist-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.8/dist-packages/scipy/stats/_distn_infrastructure.py:2604: RuntimeWarning: Degrees of freedom <= 0 for slice
  mu2hat = tmp.var()
/usr/local/lib/python3.8/dist-packages/numpy/core/_methods.py:222: RuntimeWarning: invalid value encountered in true_divide
  arrmean = um.true_divide(arrmean, div, out=arrmean, casting='unsafe',
/usr/local/lib/python3.8/dist-packages/numpy/core/_methods.py:254: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


ValueError: zero-size array to reduction operation minimum which has no identity

In [ ]:
df_make_separation(all_vals_df, 0.7, 0.1)

/usr/local/lib/python3.8/dist-packages/scipy/stats/_distn_infrastructure.py:2603: RuntimeWarning: Mean of empty slice.
  muhat = tmp.mean()
/usr/local/lib/python3.8/dist-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.8/dist-packages/scipy/stats/_distn_infrastructure.py:2604: RuntimeWarning: Degrees of freedom <= 0 for slice
  mu2hat = tmp.var()
/usr/local/lib/python3.8/dist-packages/numpy/core/_methods.py:222: RuntimeWarning: invalid value encountered in true_divide
  arrmean = um.true_divide(arrmean, div, out=arrmean, casting='unsafe',
/usr/local/lib/python3.8/dist-packages/numpy/core/_methods.py:254: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


ValueError: zero-size array to reduction operation minimum which has no identity

In [ ]:
df_make_separation(all_vals_df, 1, 0.1)

In [ ]:
df_make_separation(all_vals_df, 2, 0.1)

In [ ]:
df_make_separation(all_vals_df, 5, 0.1)

In [ ]:
df_make_separation(all_vals_df, 10, 0.1)